In [1]:
from tqdm import tqdm
import yaml
import pandas as pd
from _utils import *
from Model import Model_Class
import evaluate
import re
import torch
import copy

2024-04-13 04:04:29.318607: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-13 04:04:29.340229: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-13 04:04:29.340272: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-13 04:04:29.340821: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-13 04:04:29.344635: I tensorflow/core/platform/cpu_feature_guar

**T5 w/o Training**

In [13]:
T5_config = yaml.load(open("configs/T5_config.yml"),Loader=yaml.FullLoader)
T5_config

{'task': 'concode',
 'always_save_model': False,
 'save_last_checkpoints': False,
 'cache_path': 'cache_data',
 'load_model_path': None,
 'config_name': 'Salesforce/codet5-base',
 'model_name': 't5-base',
 'tokenizer_name': 'Salesforce/codet5-base',
 'dev_filename': 'validation_filtered.json',
 'train_filename': 'train_filtered.json',
 'test_filename': 'None',
 'lang': 'SQL',
 'data_dir': 'data',
 'output_dir': 'saved_models/',
 'summary_dir': 'summary',
 'res_dir': 'res',
 'cpu_cont': 1,
 'device': 'cuda',
 'do_eval': True,
 'do_eval_bleu': False,
 'do_test': True,
 'do_train': True,
 'summary_verbose': False,
 'log_verbose': True,
 'info_level': 1,
 'is_sample': False,
 'sample_size': 5000,
 'seed': 1234,
 'learning_rate': '5e-05',
 'start_epoch': 0,
 'num_train_epochs': 5,
 'train_batch_size': 64,
 'eval_batch_size': 64,
 'adam_epsilon': '1e-08',
 'max_source_length': 100,
 'max_target_length': 50,
 'beam_size': 1,
 'gradient_accumulation_steps': 1,
 'data_num': -1,
 'patience': 5,


In [14]:
T5_Model = Model_Class(T5_config)

04/13/2024 03:48:21 - INFO - Model -   Finish loading model [223M] from t5-base


In [6]:
answer = []
prediction = []
with open("data/testing_filtered.json") as f:
    for line in tqdm(f.readlines(),desc="Generating Predictions"):
        q1 = json.loads(line)
        prediction.append(T5_Model.generate(q1["nl"]))
        answer.append(q1["code"])

Generating Predictions:   0%|          | 0/4443 [00:00<?, ?it/s]

Generating Predictions: 100%|██████████| 4443/4443 [34:25<00:00,  2.15it/s]


In [7]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=prediction,
                        references=answer)
print(results)

04/13/2024 03:40:28 - INFO - absl -   Using default tokenizer.


{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, 'rougeLsum': 0.0}


In [5]:
T5_Model.training_loop()

04/12/2024 23:14:59 - INFO - Model -   Load cache data from cache_data/train_all.pt
/home/matt/anaconda3/envs/NLP/lib/python3.11/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
04/12/2024 23:15:00 - INFO - Model -   ***** Running training *****
04/12/2024 23:15:00 - INFO - Model -     Num examples = 272628
04/12/2024 23:15:00 - INFO - Model -     Batch size = 64
04/12/2024 23:15:00 - INFO - Model -     Batch num = 4260
04/12/2024 23:15:00 - INFO - Model -     Num epoch = 5
[0] Train loss 1.352: 100%|██████████| 4260/4260 [17:12<00:00,  4.13it/s]
04/12/2024 23:32:12 - INFO - Model -   Load cache data from cache_data/dev_all.pt
04/12/2024 23:32:12 - INFO - Model -     ***** Running ppl evaluation *****
04/12/2024 23:32:12 - INFO - Model -     Num examples =

**Load Baseline w/o Training Config**

In [2]:
Base_config = yaml.load(open("configs/Base_config.yml"),Loader=yaml.FullLoader)
Base_config

{'task': 'concode',
 'always_save_model': False,
 'save_last_checkpoints': False,
 'cache_path': 'cache_data',
 'load_model_path': None,
 'config_name': 'Salesforce/codet5-base',
 'model_name': 'Salesforce/codet5-base',
 'tokenizer_name': 'Salesforce/codet5-base',
 'dev_filename': 'validation_filtered.json',
 'train_filename': 'train_filtered.json',
 'test_filename': 'None',
 'lang': 'SQL',
 'data_dir': 'data',
 'output_dir': 'saved_models/',
 'summary_dir': 'summary',
 'res_dir': 'res',
 'cpu_cont': 1,
 'device': 'cuda',
 'do_eval': True,
 'do_eval_bleu': False,
 'do_test': True,
 'do_train': True,
 'summary_verbose': False,
 'log_verbose': True,
 'info_level': 1,
 'is_sample': False,
 'sample_size': 5000,
 'seed': 1234,
 'learning_rate': '5e-05',
 'start_epoch': 0,
 'num_train_epochs': 5,
 'train_batch_size': 64,
 'eval_batch_size': 64,
 'adam_epsilon': '1e-08',
 'max_source_length': 100,
 'max_target_length': 50,
 'beam_size': 1,
 'gradient_accumulation_steps': 1,
 'data_num': -1,
 

**Test Baseline w/o Training**

In [ ]:
Base_Model = Model_Class(Base_config)
answer = []
prediction = []
with open("data/testing_filtered.json") as f:
    for line in tqdm(f.readlines(),desc="Generating Predictions"):
        q1 = json.loads(line)
        prediction.append(Base_Model.generate(q1["nl"]))
        answer.append(q1["code"])

In [13]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=prediction,
                        references=answer)
print(results)

04/06/2024 16:41:53 - INFO - absl -   Using default tokenizer.


{'rouge1': 0.15734154154202606, 'rouge2': 0.05234838613042284, 'rougeL': 0.14547830785925475, 'rougeLsum': 0.14561331610868022}


**Load Baseline w/ Training Config**

In [5]:
config = yaml.load(open("configs/config.yml"),Loader=yaml.FullLoader)
config

{'task': 'concode',
 'always_save_model': True,
 'save_last_checkpoints': False,
 'cache_path': 'cache_data',
 'load_model_path': 'saved_models/Baseline/Baseline_Best.bin',
 'config_name': 'Salesforce/codet5-base',
 'model_name': 'Salesforce/codet5-base',
 'tokenizer_name': 'Salesforce/codet5-base',
 'dev_filename': 'validation_filtered.json',
 'train_filename': 'train_filtered.json',
 'test_filename': 'None',
 'lang': 'SQL',
 'data_dir': 'data',
 'output_dir': 'saved_models/',
 'summary_dir': 'summary',
 'res_dir': 'res',
 'cpu_cont': 1,
 'device': 'cuda',
 'do_eval': True,
 'do_eval_bleu': False,
 'do_test': True,
 'do_train': True,
 'summary_verbose': False,
 'log_verbose': True,
 'info_level': 1,
 'is_sample': False,
 'sample_size': 5000,
 'seed': 1234,
 'learning_rate': '5e-05',
 'start_epoch': 0,
 'num_train_epochs': 5,
 'train_batch_size': 64,
 'eval_batch_size': 64,
 'adam_epsilon': '1e-08',
 'max_source_length': 100,
 'max_target_length': 50,
 'beam_size': 1,
 'gradient_accumu

**Load & Train Model**

In [3]:
Model = Model_Class(config)
train_base = input("Would you like to train the Base_Model (y/n): ")
if train_base == 'y':
    Model.training_loop()

/home/matt/anaconda3/envs/NLP/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
04/06/2024 07:42:22 - INFO - Model -   Finish loading model [223M] from Salesforce/codet5-base
04/06/2024 07:42:22 - INFO - Model -   Reload model from saved_models/Baseline/Base.bin
04/06/2024 07:42:34 - INFO - Model -   Load cache data from cache_data/train_all.pt
/home/matt/anaconda3/envs/NLP/lib/python3.11/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.

**Test Baseline w/ Training**

In [ ]:
answer = []
prediction = []
with open("data/testing_filtered.json") as f:
    for line in tqdm(f.readlines(),desc="Generating Predictions"):
        q1 = json.loads(line)
        prediction.append(Model.generate(q1["nl"]))
        answer.append(q1["code"])

In [9]:
rouge = evaluate.load('rouge')
results = rouge.compute(predictions=prediction,
                        references=answer)
print(results)

04/06/2024 15:50:48 - INFO - absl -   Using default tokenizer.


{'rouge1': 0.8636640033742007, 'rouge2': 0.8019490870939264, 'rougeL': 0.8500682335775959, 'rougeLsum': 0.8498248334695415}


**Setup and Apply Dora**

In [2]:
Dora_Config = yaml.load(open("configs/Dora_config.yml"),Loader=yaml.FullLoader)
Dora_Config

{'task': 'concode',
 'always_save_model': True,
 'save_last_checkpoints': False,
 'cache_path': 'cache_data',
 'load_model_path': None,
 'config_name': 'Salesforce/codet5-base',
 'model_name': 'Salesforce/codet5-base',
 'tokenizer_name': 'Salesforce/codet5-base',
 'dev_filename': 'validation_filtered.json',
 'train_filename': 'train_filtered.json',
 'test_filename': 'None',
 'lang': 'SQL',
 'data_dir': 'data',
 'output_dir': 'saved_models/',
 'summary_dir': 'summary',
 'res_dir': 'res',
 'cpu_cont': 1,
 'device': 'cuda',
 'do_eval': True,
 'do_eval_bleu': False,
 'do_test': True,
 'do_train': True,
 'summary_verbose': False,
 'log_verbose': True,
 'info_level': 1,
 'is_sample': False,
 'sample_size': 5000,
 'seed': 1234,
 'learning_rate': '5e-05',
 'start_epoch': 0,
 'num_train_epochs': 20,
 'train_batch_size': 64,
 'eval_batch_size': 64,
 'adam_epsilon': '1e-08',
 'max_source_length': 100,
 'max_target_length': 50,
 'beam_size': 1,
 'gradient_accumulation_steps': 1,
 'data_num': -1,
 

In [3]:
Dora_Model = Model_Class(Dora_Config)

/home/matt/anaconda3/envs/NLP/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
04/13/2024 03:57:57 - INFO - Model -   Finish loading model [223M] from Salesforce/codet5-base


In [4]:
from peft import get_peft_model, LoraConfig, TaskType
# Hyperparameters
r = 256
lora_alpha = 512 #2 times r
lora_dropout = 0.1
temp_model = Dora_Model.model

peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, 
                         inference_mode=False, 
                         r=r, 
                         lora_alpha=lora_alpha, 
                         lora_dropout=lora_dropout,
                         use_dora=True)

temp_model = get_peft_model(temp_model, peft_config)
print("Original:")
total_param = sum(p.numel() for p in Dora_Model.model.parameters())
print(f"trainable params: {total_param} || all params: {total_param} || trainable%: 100")
print("-"*80)
print("Model w/ Dora:")
temp_model.print_trainable_parameters()

Dora_Model.set_model(temp_model)

Original:
trainable params: 251248896 || all params: 251248896 || trainable%: 100
--------------------------------------------------------------------------------
Model w/ Dora:
trainable params: 28,366,848 || all params: 251,248,896 || trainable%: 11.290337371273464


**Train Dora Model**

In [ ]:
train_exp = input("Would you like to train the new Model (y/n): ")
if train_exp == 'y':
    Dora_Model.training_loop()

**Test Dora Model**

In [7]:
answer = []
prediction = []
with open("data/testing_filtered.json") as f:
    for line in tqdm(f.readlines(),desc="Generating Predictions"):
        q1 = json.loads(line)
        prediction.append(Dora_Model.generate(q1["nl"]))
        answer.append(q1["code"])

rouge = evaluate.load('rouge')
results = rouge.compute(predictions=prediction,
                        references=answer)
print(results)

Generating Predictions:   0%|          | 0/4443 [00:00<?, ?it/s]

Generating Predictions: 100%|██████████| 4443/4443 [42:18<00:00,  1.75it/s]
04/08/2024 05:11:48 - INFO - absl -   Using default tokenizer.


{'rouge1': 0.822827901409901, 'rouge2': 0.7356028427074228, 'rougeL': 0.8017589341657259, 'rougeLsum': 0.8017071706924369}


**Make 3 different training datasets**

**1. Masked Span Prediction**

In [14]:
Masked_span = []
with open("data/train_filtered.json","r") as f:
    for line in tqdm(f.readlines()):
        code = json.loads(line)['code']
        Masked_span.append({"nl": code, "code": code})

100%|██████████| 314887/314887 [00:00<00:00, 569715.60it/s]


In [15]:
import re
import numpy as np
import copy

for x in tqdm(Masked_span):
    out_list = re.split('([^a-zA-Z0-9_])', x["nl"])

    num_mask = np.random.randint(1, 2+int(len(out_list)/4))
    mask_values = np.random.choice(range(len(out_list)),num_mask,replace=False)

    for i, word in enumerate([out_list[i] for i in mask_values]):
        if len(word) > 3:
            out_list[out_list.index(word)] = f"Mask{i}"
    x["nl"] = ''.join(out_list)

100%|██████████| 314887/314887 [00:04<00:00, 63948.76it/s]


In [19]:
with open("data/training_Masked_Span_Prediction.json", "w") as f:
    for x in tqdm(Masked_span[:-5000]):
        json.dump(x,f,indent=None,default=str)
        f.write("\n")

with open("data/validation_training_Masked_Span_Prediction.json", "w") as f:
    for x in tqdm(Masked_span[-5000:]):
        json.dump(x,f,indent=None,default=str)
        f.write("\n")

100%|██████████| 5000/5000 [00:00<00:00, 178885.99it/s]


**2. Identifier Tagging**

In [154]:
Identifier_Tagging = []
with open("data/train_filtered.json","r") as f:
    for line in tqdm(f.readlines()):
        code = json.loads(line)['code']
        Identifier_Tagging.append({"nl": code})

100%|██████████| 314887/314887 [00:00<00:00, 546922.88it/s]


In [157]:
keywords = ["ADD","CONSTRAINT","ALL","ALTER","COLUMN","TABLE","AND","ANY","AS","ASC","BACKUP","DATABASE","BETWEEN","CASE","CHECK","CREATE","INDEX","OR","REPLACE","VIEW","PROCEDURE","UNIQUE","DEFAULT","DELETE","DESC","DISTINCT","DROP","EXEC","EXISTS","FOREIGN","KEY","FROM","FULL","OUTER","JOIN","GROUP","BY","HAVING","IN","INNER","INSERT","INTO","SELECT","IS","NULL","NOT","LEFT","LIKE","LIMIT","ORDER","PRIMARY","RIGHT","ROWNUM","TOP","SET","TRUNCATE","UNION","ALL","UPDATE","VALUES","WHERE"]
for i in tqdm(Identifier_Tagging):
    temp_identifiers = re.split("\\W+",i["nl"])
    out_list = re.split('([^a-zA-Z0-9_])', i["nl"])

    i["code"] = [1 if x in temp_identifiers and x not in keywords else 0
    for x in out_list if x.strip()]
    i["code"] = str(i["code"])

  0%|          | 0/314887 [00:00<?, ?it/s]

100%|██████████| 314887/314887 [00:02<00:00, 110220.86it/s]


In [158]:
with open("data/training_identifier_tagging.json", "w") as f:
    for x in tqdm(Identifier_Tagging[:-5000]):
        json.dump(x,f,indent=None,default=str)
        f.write("\n")

with open("data/validation_training_identifier_tagging.json", "w") as f:
    for x in tqdm(Identifier_Tagging[-5000:]):
        json.dump(x,f,indent=None,default=str)
        f.write("\n")

100%|██████████| 5000/5000 [00:00<00:00, 201924.94it/s]


**3. Masked Identifier Prediction** 

In [22]:
# "training_Masked_Identifier_Prediciton.json")
Masked_Identifier_Prompt = "{nl}\n### Masked:\n{code}"
Masked_Identifier_Tagging = []
with open("data/train_filtered.json","r") as f:
    for line in tqdm(f.readlines()):
        Masked_Identifier_Tagging.append(json.loads(line))

100%|██████████| 314887/314887 [00:00<00:00, 543970.61it/s]


In [23]:
keywords = ["ADD","CONSTRAINT","ALL","ALTER","COLUMN","TABLE","AND","ANY","AS","ASC","BACKUP","DATABASE","BETWEEN","CASE","CHECK","CREATE","INDEX","OR","REPLACE","VIEW","PROCEDURE","UNIQUE","DEFAULT","DELETE","DESC","DISTINCT","DROP","EXEC","EXISTS","FOREIGN","KEY","FROM","FULL","OUTER","JOIN","GROUP","BY","HAVING","IN","INNER","INSERT","INTO","SELECT","IS","NULL","NOT","LEFT","LIKE","LIMIT","ORDER","PRIMARY","RIGHT","ROWNUM","TOP","SET","TRUNCATE","UNION","ALL","UPDATE","VALUES","WHERE"]
for i in tqdm(Masked_Identifier_Tagging):
    temp_identifiers = re.split("\\W+",i["code"])
    out_list = re.split('([^a-zA-Z0-9_])', i["code"])
    Masked_code = [f"Mask{pos}" if x in temp_identifiers and x not in keywords and x.strip() else x
    for pos, x in enumerate(out_list)]
    Masked_code = "".join(i["code"])
    i["nl"] = Masked_Identifier_Prompt.format_map({"nl":i["nl"],"code":Masked_code})

100%|██████████| 314887/314887 [00:04<00:00, 75078.08it/s]


In [24]:
with open("data/training_Masked_Identifier_Prediciton.json", "w") as f:
    for x in tqdm(Masked_Identifier_Tagging[:-5000]):
        json.dump(x,f,indent=None,default=str)
        f.write("\n")
with open("data/Validation_training_Masked_Identifier_Prediciton.json", "w") as f:
    for x in tqdm(Masked_Identifier_Tagging[-5000:]):
        json.dump(x,f,indent=None,default=str)
        f.write("\n")

100%|██████████| 5000/5000 [00:00<00:00, 142720.68it/s]


**Experiment Model**

In [2]:
Experiment_Config = yaml.load(open("configs/Experiment_config.yml"),Loader=yaml.FullLoader)
Experiment_Config

{'task': 'concode',
 'always_save_model': True,
 'save_last_checkpoints': False,
 'cache_path': 'cache_data',
 'load_model_path': None,
 'config_name': 'Salesforce/codet5-base',
 'model_name': 'Salesforce/codet5-base',
 'tokenizer_name': 'Salesforce/codet5-base',
 'dev_filename': 'validation_training_Masked_Span_Prediction.json',
 'train_filename': 'training_Masked_Span_Prediction.json',
 'test_filename': 'None',
 'lang': 'SQL',
 'data_dir': 'data',
 'output_dir': 'saved_models/',
 'summary_dir': 'summary',
 'res_dir': 'res',
 'cpu_cont': 1,
 'device': 'cuda',
 'do_eval': True,
 'do_eval_bleu': False,
 'do_test': True,
 'do_train': True,
 'summary_verbose': False,
 'log_verbose': True,
 'info_level': 1,
 'is_sample': False,
 'sample_size': 5000,
 'seed': 1234,
 'learning_rate': '5e-05',
 'start_epoch': 0,
 'num_train_epochs': 5,
 'train_batch_size': 64,
 'eval_batch_size': 64,
 'adam_epsilon': '1e-08',
 'max_source_length': 100,
 'max_target_length': 50,
 'beam_size': 1,
 'gradient_acc

In [3]:
Experiment_Model = Model_Class(Experiment_Config)

/home/matt/anaconda3/envs/NLP/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
04/13/2024 04:04:35 - INFO - Model -   Finish loading model [223M] from Salesforce/codet5-base


In [4]:
from peft import get_peft_model, LoraConfig, TaskType
# Hyperparameters
r = 256
lora_alpha = 512 #2 times r
lora_dropout = 0.1
temp_model = Experiment_Model.model

peft_config = LoraConfig(task_type=TaskType.SEQ_CLS, 
                         inference_mode=False, 
                         r=r, 
                         lora_alpha=lora_alpha, 
                         lora_dropout=lora_dropout,
                         use_dora=True)

temp_model = get_peft_model(temp_model, peft_config)
print("Original:")
total_param = sum(p.numel() for p in Experiment_Model.model.parameters())
print(f"trainable params: {total_param} || all params: {total_param} || trainable%: 100")
print("-"*80)
print("Model w/ Dora:")
temp_model.print_trainable_parameters()

Experiment_Model.set_model(temp_model)

Original:
trainable params: 251248896 || all params: 251248896 || trainable%: 100
--------------------------------------------------------------------------------
Model w/ Dora:
trainable params: 28,366,848 || all params: 251,248,896 || trainable%: 11.290337371273464


**Training Masked Span Prediction**

In [ ]:
train_exp = input("Would you like to train the new Model (y/n): ")
if train_exp == 'y':
    Experiment_Model.training_loop(split_tags=["train_MSP_wo_Dora","dev_MSP_wo_Dora"])

In [4]:
#Load Saved
Experiment_Model.load_model(model=Experiment_Model.model, path="saved_models/msp_wo_dora_model.bin")

04/10/2024 03:20:24 - INFO - Model -   Finish loading model [223M] from Salesforce/codet5-base
04/10/2024 03:20:24 - INFO - Model -   Reload model from saved_models/msp_wo_dora_model.bin


**Identifier Tagging**

Modify the model so that the last layer of the encoder transformer heads are provided as output from the model.

Attach the encoder heads to the desired task / loss.

Train the modified model using the new loss based only on the encoder heads and not any part of the decoder, providoing task input text to the encoder and (if they really want to minimize code changes) pass an empty sequence to the decoder.

In [5]:
# Change training data
Experiment_Model.__setattr__("train_filename","data/training_identifier_tagging.json")
Experiment_Model.__setattr__("dev_filename","data/validation_training_identifier_tagging.json")

In [6]:
list_of_weights = []
for x in Experiment_Model.model.decoder.parameters():
    x.requires_grad = False
    list_of_weights.append(x)

In [ ]:
Experiment_Model.training_loop(split_tags=["train_it_wo_dora","dev_it_wo_dora"])

In [9]:
new_list_of_weights = []
for x in Experiment_Model.model.decoder.parameters():
    new_list_of_weights.append(x)
    x.requires_grad = True

print(list_of_weights == new_list_of_weights)

True


In [4]:
# Experiment_Model.save_model_state("v2_it_model_wo_dora.bin")
# Experiment_Model.load_model(model=Experiment_Model.model, path="saved_models/v2_it_model_wo_dora.bin")

04/10/2024 04:11:47 - INFO - Model -   Finish loading model [223M] from Salesforce/codet5-base
04/10/2024 04:11:47 - INFO - Model -   Reload model from saved_models/v2_it_model_wo_dora.bin


**Masked Identifier Prediction**

In [6]:
Experiment_Model.__setattr__("train_filename","data/training_Masked_Identifier_Prediciton.json")
Experiment_Model.__setattr__("dev_filename","data/Validation_training_Masked_Identifier_Prediciton.json")

In [ ]:
Experiment_Model.training_loop(split_tags=["train_mip_wo_dora","dev_mip_wo_dora"])

**Train on default data**

In [5]:
Experiment_Model.__setattr__("dev_filename","data/train_filtered.json")
Experiment_Model.__setattr__("train_filename","data/validation_filtered.json")

In [ ]:
Experiment_Model.training_loop()

In [10]:
Experiment_Model.save_model_state("final_experiment_model.bin")

**Testing Experiment**

In [11]:
answer = []
prediction = []
with open("data/testing_filtered.json") as f:
    for line in tqdm(f.readlines(),desc="Generating Predictions"):
        q1 = json.loads(line)
        prediction.append(Experiment_Model.generate(q1["nl"]))
        answer.append(q1["code"])

rouge = evaluate.load('rouge')
results = rouge.compute(predictions=prediction,
                        references=answer)
print(results)

Generating Predictions: 100%|██████████| 4443/4443 [24:21<00:00,  3.04it/s]
04/10/2024 06:33:41 - INFO - absl -   Using default tokenizer.


{'rouge1': 0.8947285696912889, 'rouge2': 0.8410073084335176, 'rougeL': 0.8814300390326623, 'rougeLsum': 0.8812268044482586}


## Evaluation Metric

In [14]:
from torchmetrics.text import CHRFScore

In [29]:
chrf = CHRFScore(n_char_order=1, 
                 n_word_order=2,
                 lowercase=True,
                 whitespace=False)
chrf(Model.generate(q1["nl"]),q1["code"])

tensor(0.1609)

**Tests**

In [40]:
PROMPT_DICT = """
### Question
Write an SQL query that answers this question:
{question}

### Context
The query will run on a database with the following schema:
{context}
"""
q1 = {"nl": "\n    ### Question\n    Write an SQL query that answers this question:\n    What is the highest Byes by Anakie who has an Against smaller than 2275?\n\n    ### Context\n    The query will run on a database with the following schema:\n    CREATE TABLE table_name_14 (\n    byes INTEGER,\n    geelong_dfl VARCHAR,\n    against VARCHAR\n)\n    ", "code": "SELECT MAX(byes) FROM table_name_14 WHERE geelong_dfl = \"anakie\" AND against < 2275"}

print(q1["nl"])
print("-"*20)
print(q1["code"])
print("-"*20)
print(Model.generate(q1["nl"]))


    ### Question
    Write an SQL query that answers this question:
    What is the highest Byes by Anakie who has an Against smaller than 2275?

    ### Context
    The query will run on a database with the following schema:
    CREATE TABLE table_name_14 (
    byes INTEGER,
    geelong_dfl VARCHAR,
    against VARCHAR
)
    
--------------------
SELECT MAX(byes) FROM table_name_14 WHERE geelong_dfl = "anakie" AND against < 2275
--------------------
SELECT MAX(byes) FROM table_name_14 WHERE geelong_dfl = "anakie" AND against < 2275
